In [1]:
import pandas as pd
import re
import unicodedata
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# ======= Étape 1 : Chargement et nettoyage des données ======= #
# Charger les données
data = pd.read_csv("final_output.csv")

# Nettoyer les colonnes inutiles
colonnes_a_supprimer = ['Latitude', 'Longitude', 'Periodes_regroupees', 'Covid19_mesures_specifiques', 
                        'Createur_de_la_donnee', 'SIT_diffuseur', 'Date_de_mise_a_jour', 'URI_ID_du_POI']
data = data.drop(columns=colonnes_a_supprimer, errors='ignore')

# Supprimer les lignes avec valeurs manquantes
data = data.dropna(subset=['Nom_du_POI', 'Description', 'Code_postal_et_commune'])

# Fonction pour nettoyer le texte
def nettoyer_texte(texte):
    texte = unicodedata.normalize('NFKD', str(texte))  # Enlever accents
    texte = re.sub(r'[^a-zA-Z0-9\s.,-]', '', texte)   # Garder lettres, chiffres, espace, ., , et -
    texte = re.sub(r'\s+', ' ', texte)  # Remplacer les espaces multiples
    return texte.strip()

data['Nom_du_POI'] = data['Nom_du_POI'].apply(nettoyer_texte)
data['Description'] = data['Description'].apply(nettoyer_texte)

# Supprimer les doublons
data = data.drop_duplicates(subset=['Nom_du_POI', 'Code_postal_et_commune', 'Description'])

# Créer des prompts structurés
def transformer_donnees(row):
    return f"Destination : {row['Code_postal_et_commune'].split('#')[-1]}\n" \
           f"Activité : {row['Nom_du_POI']}\n" \
           f"Catégorie : {row['Categories_de_POI'].split('#')[-1]}\n" \
           f"Description : {row['Description']}\n"

data["prompt"] = data.apply(transformer_donnees, axis=1)





ModuleNotFoundError: No module named 'pandas'

In [ ]:
# ======= Étape 2 : Préparer les données pour GPT-2 ======= #
dataset = Dataset.from_dict({"text": data["prompt"].tolist()})

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Ajoute un token de fin

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [2]:
# ======= Étape 3 : Configuration de l'entraînement ======= #
model = GPT2LMHeadModel.from_pretrained("gpt2")
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./output",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=50
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# ======= Étape 4 : Entraînement ======= #
print("Début de l'entraînement...")
trainer.train()

# ======= Étape 5 : Génération de texte ======= #
print("\nTest de génération de recommandations :")
prompt = "Destination : Lyon\nActivité recommandée :"
input_ids = tokenizer.encode(prompt, return_tensors="pt")
output = model.generate(input_ids, max_length=100, num_return_sequences=1)
resultat = tokenizer.decode(output[0], skip_special_tokens=True)
print(resultat)

NameError: name 'GPT2LMHeadModel' is not defined

548137    Destination : Vendresse\nActivité : Fete de la...
Name: prompt, dtype: object

548137    Destination : Vendresse\nActivité : Fete de la...
Name: prompt, dtype: object